In [8]:
from ipywidgets import *
from IPython.display import display, clear_output
import os
import json
import folium
import pandas as pd
import numpy as np
import boto3
import branca.colormap as cm
import numpy as np
import config as cfg

In [44]:
#creates function which dictates outputs based off textbox intput and button "clicks"
def btn_execute_app(obj): 
    with output:
        clear_output()
        
        if len(destination_loc.value) < 1:
            raise ValueError("Please enter a value for your destination.")
        else:
            pass
        
        if len(commute_time.value) < 1:
            raise ValueError("Please enter a value for your desired commute time.")
        else:
            pass
        
        if len(min_income_input_box.value) < 1:
            raise ValueError("Please enter a value for your minimum yearly income.")
        else:
            pass
        
        if len(max_income_input_box.value) < 1:
            raise ValueError("Please enter a value for your maximum yearly income.")
        else:
            pass
        
        if commute_time.value.isdecimal() == False:
            raise ValueError("Your commute time input is not valid, please enter a whole number.")
        else:
            pass
        
        if min_income_input_box.value.isdecimal() == False:
            raise ValueError("Your minimum income value input is not valid, please enter a whole number.")
        else:
            pass
        
        if max_income_input_box.value.isdecimal() == False:
            raise ValueError("Your maximum income value input is not valid, please enter a whole number.")
        else:
            pass     

        def write_census_data_to_csv(s0801_path, s1902_path, tract_shapes_path):
            """Writes the data downloaded directly from ACS to TIGER shapefiles.

            Reads in data from .csv format as downloaded from the American Community
            Survey (ACS) website, then filters to variables of interest and saves. In
            this case the two tables are s0801 and s1902, which contain basic
            socioeconomic and commute-related variables. Data was downloaded at the
            census tract level for the state of Washington. The data is saved in .csv
            format to be used in the Folium map.

            Args:
                s0801_path: A string path to the location of the raw s0801 data, not
                    including file type ending (.csv).
                s1902_path: A string path to the location of the raw s1902 data, not
                    including file type ending (.csv).
                tract_shapes_path: A string path to the geojson TIGER shapefile as
                    downloaded from the ACS, containing polygon data for census tracts
                    in the state of Washington.

            Returns:
                1 after writing the combined datasets to a *_tmp file in the same folder
                as the TIGER shapefiles.
            """
            # Read in the two tables that were taken from the ACS data portal website
            s0801_df = pd.read_csv(f"{s0801_path}.csv")
            s1902_df = pd.read_csv(f"{s1902_path}.csv")
            # Filter each table to only variables of interest, make more descriptive
            commuters_df = s0801_df[[
                'GEO_ID', 'NAME',
                'S0801_C01_001E', 'S0801_C01_009E']]
            commuters_df.columns = [
                'GEO_ID', 'NAME',
                'total_workers', 'workers_using_transit']
            commuters_df = commuters_df.loc[1:len(commuters_df), :]
            commuters_df['GEO_ID'] = commuters_df['GEO_ID'].str[-11:]

            # Repeat for s1902
            households_df = s1902_df[[
                'GEO_ID', 'NAME', 'S1902_C01_001E',
                'S1902_C03_001E', 'S1902_C02_008E',
                'S1902_C02_020E', 'S1902_C02_021E']]
            households_df.columns = [
                'GEO_ID', 'NAME', 'total_households',
                'mean_income', 'percent_w_assistance',
                'percent_white', 'percent_black_or_african_american']
            households_df = households_df.loc[1:len(households_df), :]
            households_df['GEO_ID'] = households_df['GEO_ID'].str[-11:]

            # Combine datasets on their census tract ID and write to .csv file
            final_df = pd.merge(commuters_df, households_df, on='GEO_ID').drop(columns=['NAME_x', 'NAME_y'])
            final_df.to_csv(f"{tract_shapes_path}_tmp.csv", index=False)
            return 1
        
        def write_speeds_to_map_segments(speed_lookup, segment_file):
            """Creates a _tmp geojson file with speed data downloaded from dynamodb.

            Loads the segments generated from initialize_db.py and adds speeds to them
            based on the specified dictionary. Writes a new *_tmp geojson file that will
            be loaded by the Folium map and color coded based on segment average speed.

            Args:
                speed_lookup: A Dictionary object with (route id, local_express_code)
                    keys and average speed data to be plotted by Folium.
                segment_file: A string path to the geojson file generated by
                    initialize_db.py that contains route coordinate data.

            Returns:
                A list containing the average speed of each segment that was
                successfully paired to a route (and will be plotted on the map).
            """
            if isinstance(speed_lookup, dict):
                pass
            else:
                raise TypeError('Speed lookup must be a dictionary')

            # Read route geojson, add property for avg speed, keep track of all speeds
            speeds = np.ones(0)
            with open(f"{segment_file}.geojson", 'r') as shapefile:
                kcm_routes = json.load(shapefile)
            # Check if each geojson feature has a speed in the database
            for feature in kcm_routes['features']:
                route_id = feature['properties']['ROUTE_ID']
                local_express_code = feature['properties']['LOCAL_EXPR']
                if (route_id, local_express_code) in speed_lookup.keys():
                    speed = speed_lookup[(route_id, local_express_code)]['avg_speed_m_s']
                    feature['properties']['AVG_SPEED_M_S'] = speed
                    speeds = np.append(speeds, speed)
                else:
                    feature['properties']['AVG_SPEED_M_S'] = 0
            with open(f"{segment_file}_w_speeds_tmp.geojson", 'w+') as new_shapefile:
                json.dump(kcm_routes, new_shapefile)
            return speeds
        
        def generate_folium_map(segment_file, census_file, colormap):
            """Draws together speed/socioeconomic data to create a Folium map.

            Loads segments with speed data, combined census data, and the colormap
            generated from the list of speeds to be plotted. Plots all data sources on
            a new Folium Map object centered on Seattle, and returns the map.

            Args:
                segment_file: A string path to the geojson file generated by
                    write_speeds_to_map_segments that should contain geometry as well
                    as speed data.
                census_file: A string path to the geojson file generated by
                    write_census_data_to_csv that should contain the combined s0801 and
                    s1902 tables.
                colormap: A Colormap object that describes what speeds should be mapped
                    to what colors.

            Returns:
                A Folium Map object containing the most up-to-date speed data from the
                dynamodb.
            """
            # Read in route shapefile and give it the style function above
            kcm_routes = folium.GeoJson(
                f"{segment_file}_w_speeds_tmp.geojson",
                style_function=lambda feature: {
                    'color': colormap(feature['properties']['AVG_SPEED_M_S']),
                    'weight': 2})


            # Read in the census data/shapefile and create a choropleth based on income
            seattle_tracts_df = pd.read_csv(f"{census_file}_tmp.csv")
            seattle_tracts_df['GEO_ID'] = seattle_tracts_df['GEO_ID'].astype(str)
            seattle_tracts_df['mean_income'] = pd.to_numeric(\
                                                 seattle_tracts_df['mean_income'],\
                                                 errors='coerce')
            
            # mde the number smaller so the legend would look better
            seattle_tracts_df['mean_income'] = seattle_tracts_df['mean_income']/1000
            
            seattle_tracts_df = seattle_tracts_df.dropna()    
                     
            seattle_tracts = folium.Choropleth(
                geo_data=f"{census_file}.geojson",
                name='Socioeconomic Data',
                data=seattle_tracts_df,
                columns=['GEO_ID', 'mean_income'], 
                # added thresholding values for coloring based off widget inputs
                threshold_scale=[seattle_tracts_df["mean_income"].min()-1, \
                                 int(min_income_input_box.value)/1000, \
                                 int(max_income_input_box.value)/1000, \
                                 seattle_tracts_df["mean_income"].max()+1],
                key_on='feature.properties.GEOID10',
                fill_color="PuBuGn",
                #fill_color='PuBu',
                fill_opacity=0.7,
                line_opacity=0.4,
                legend_name='mean income (1000s)')
            
            #Create folium marker based off destination
            dest_lat = float(destination_loc.value.split(",")[0])
            dest_long = float(destination_loc.value.split(" ")[1])            
            dest_marker = folium.Marker(
                location=[dest_lat, dest_long], 
                popup="Destination", 
                icon=folium.Icon(color="green", icon="info-sign"))
            
            # Draw map using the speeds and census data
            f_map = folium.Map(
                location=[47.606209, -122.332069],
                zoom_start=11,
                prefer_canvas=True)
            seattle_tracts.add_to(f_map)
            kcm_routes.add_to(f_map)
            dest_marker.add_to(f_map)
            colormap.caption = 'Average Speed (m/s)'
            colormap.add_to(f_map)
            folium.LayerControl().add_to(f_map)
            return f_map
        
        def save_and_view_map(f_map, output_file):
            """Writes Folium Map to an output file and prints the path to the terminal.

            Saves the specified Folium Map to the specified location. File is saved as
            an .html file. The user can then open the path in any browser to display
            it.

            Args:
                f_map: A Folium Map object to be written.
                output_file: A string path to the location where the Folium Map should
                    be saved. Include file type ending (.html).

            Returns:
                1 when done writing and printing the Folium map .html file.
            """
            if output_file[-5:] == '.html':
                pass
            else:
                raise TypeError('output file must be an html')

            f_map.save(f"{output_file}")
            current_directory = os.getcwd()
            print("Map saved, please copy this file path into any browser: "+\
                  "file://"+
                  current_directory+'/'+\
                  f"{output_file}")
            return 1
        
        def connect_to_dynamo_table(table_name):
            """Connects to the dynamodb table specified using details from config.py.

            Uses the AWS login information stored in config.py to attempt a connection
            to dynamodb using the boto3 library, then creates a connection to the
            specified table.

            Args:
                table_name: The name of the table on the dynamodb resource to connect.

            Returns:
                A boto3 Table object pointing to the dynamodb table specified.
            """
            dynamodb = boto3.resource(
                'dynamodb',
                region_name=cfg.REGION,
                aws_access_key_id=cfg.ACCESS_ID,
                aws_secret_access_key=cfg.ACCESS_KEY)
            table = dynamodb.Table(table_name)
            return table

        def dump_table(dynamodb_table):
            """Downloads the contents of a dynamodb table and returns them as a list.

            Iterates through the contents of a dynamodb scan() call, which returns a
            LastEvaluatedKey until there are no results left in the scan. Appends each
            chunk of data returned by scan to an array for further use.

            Args:
                dynamodb_table: A boto3 Table object from which all data will be read
                    into memory and returned.

            Returns:
                A list of items downloaded from the dynamodb table. In this case, each
                item is a bus route as generated in initialize_db.py.
            """
            result = []
            response = dynamodb_table.scan()
            result.extend(response['Items'])
            while 'LastEvaluatedKey' in response.keys():
                response = dynamodb_table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
                result.extend(response['Items'])
            return result

        def table_to_lookup(table):
            """Converts the contents of a dynamodb table to a dictionary for reference.

            Uses dump_table to download the contents of a specified table, then creates
            a route lookup dictionary where each key is (route id, express code) and
            contains elements for avg_speed, and historic_speeds.

            Args:
                table: A boto3 Table object from which all data will be read
                    into memory and returned.

            Returns:
                A dictionary with (route id, segment id) keys and average speed (num),
                historic speeds (list), and local express code (str) data.
            """
            # Put the data in a dictionary to reference when adding speeds to geojson
            items = dump_table(table)
            route_lookup = {}
            for item in items:
                if 'avg_speed_m_s' in item.keys():
                    route_id = int(item['route_id'])
                    local_express_code = item['local_express_code']
                    route_lookup[(route_id, local_express_code)] = {
                        'avg_speed_m_s': float(item['avg_speed_m_s']),
                        'historic_speeds': item['historic_speeds']
                    }
            return route_lookup

        def main_function(
                table_name,
                s0801_file,
                s1902_file,
                segment_file,
                census_file):
            """Combines ACS data, downloads speed data, and plots map of results.

            Build the final map by first preparing ACS and dynamodb data, then plotting
            the data using the Folium library and save it to a .html file and open with
            the user's web browser.

            Args:
                table_name: The name of the dynamodb table containing speed data.
                s0801_file: A string path to the location of the raw s0801 data, not
                    including file type ending (.csv).
                s1902_file: A string path to the location of the raw s1902 data, not
                    including file type ending (.csv).
                segment_file: A string path to the geojson file generated by
                    initialize_db.py that contains route coordinate data.
                census_file: A string path to the geojson TIGER shapefile as
                    downloaded from the ACS, containing polygon data for census tracts
                    in the state of Washington.

            Returns:
                1 when done writing and opening the Folium map .html file.
            """
            # Combine census tract data from multiple ACS tables for Seattle
            print("Modifying and writing census data...")
            write_census_data_to_csv(s0801_file, s1902_file, census_file)

            # Connect to dynamodb
            print("Connecting to dynamodb...")
            table = connect_to_dynamo_table(table_name)

            # Query the dynamoDB for all speed data
            print("Getting speed data from dynamodb...")
            speed_lookup = table_to_lookup(table)

            print("Writing speed data to segments for visualization...")
            speeds = write_speeds_to_map_segments(speed_lookup, segment_file)

            # Create the color mapping for speeds
            print("Generating map...")
            linear_cm = cm.LinearColormap(
                ['red', 'yellow', 'green'],
                vmin=0.0,
                vmax=np.ceil(np.percentile(speeds[speeds > 0.0], 95)))

            f_map = generate_folium_map(segment_file, census_file, linear_cm)
            print("Saving map...")
            save_and_view_map(f_map, 'output_map.html')
            return 1
        
        main_function(table_name='KCM_Bus_Routes',\
                         s0801_file='../data/s0801',\
                         s1902_file='../data/s1902',\
                         segment_file='../data/kcm_routes',\
                         census_file='../data/seattle_census_tracts_2010')

#creates destination input box
destination_loc = widgets.Text( 
    value="47.606209, -122.332069",
    placeholder='Enter Destination Location in "lat, long"',
    description='Destination: ',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=Layout(width="380px", height="auto") 
)

#creates commute time input box
commute_time = widgets.Text(
    placeholder='Enter maximum desired commute time (mins)',
    description='Commute Time: ',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=Layout(width="380px", height="auto")
)

# creates minimum desired rent input box
min_income_input_box = widgets.Text(
    placeholder='Enter minimum yearly income ($)',
    description='Income Minimum:',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=Layout(width="380px", height="auto")
)

# creates maximum desired rent input box
max_income_input_box = widgets.Text(
    placeholder='Enter maximum yearly income ($)',
    description='Income Maximum:',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=Layout(width="380px", height="auto") # "40px"
)

# creates a clickable button to execute a fucntion based off inputs 
app_btn = widgets.Button(description='Execute Search', layout=Layout(width="380px", height="auto"))

# function which executes upon app_button being clicked executes tests/verifications 
## as well as filter and output the map
    
app_btn.on_click(btn_execute_app)

# combines all input boxes and widget together
input_box = VBox([destination_loc, commute_time, min_income_input_box, max_income_input_box, app_btn])

# assigns an output widget to display the output from the executed function and allow for overwriting
output = widgets.Output()

display(input_box)
display(output)

Output()